**SQLite Database**

In [1]:
import sqlite3

In [2]:
# create a connection to SQLite DB
conn = sqlite3.connect("test_najeeb.db")

In [3]:
# create a cursor to help us execute the SQL commands
cursor = conn.cursor()

In [4]:
# create a table
# test_sqlite : test_code, test_name

cursor.execute(
"""
CREATE TABLE IF NOT EXISTS test_sqlite (
  test_code INTEGER PRIMARY KEY,
  test_name TEXT NOT NULL
)
"""
)


In [5]:
# insert some data
cursor.execute(
    "INSERT INTO test_sqlite (test_name) VALUES (?)", ('RDBMS',)
)



In [6]:
cursor.execute(
    "INSERT INTO test_sqlite (test_name) VALUES (?)", ('TimeSeries',)
)


In [7]:
# select records

cursor.execute("SELECT * FROM test_sqlite")

In [8]:
rows = cursor.fetchall()
rows

[(1, 'RDBMS'), (2, 'TimeSeries')]

In [9]:
conn.commit()

In [10]:
conn.close()

**Using SQLite as vector storage**

In [11]:
# vector = array of numbers -> numpy arrays -> [1.2, 2,5, 3,7]
# store the information into a bytes format

In [12]:
import numpy as np

In [13]:
conn = sqlite3.connect("test_vectors.db")

In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
  vector_id INTEGER PRIMARY KEY,
  vector BLOB NOT NULL
)
"""
)

In [16]:
vect_rdbms = np.array([1.3, 3.5, 2.2, 0.9])
vect_time = np.array([2.8, 1.6, 3.8, 2.2])

In [17]:
vect_rdbms

array([1.3, 3.5, 2.2, 0.9])

In [18]:
type(vect_rdbms)

numpy.ndarray

In [19]:
vect_rdbms.tobytes()

b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'

In [20]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_rdbms.tobytes()),))

In [21]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_time.tobytes()),))

In [22]:
cursor.execute("SELECT * FROM vectors")

In [23]:
rows = cursor.fetchall()
rows

[(1,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

# Deserialization

In [ ]:
# np.frombuffer

In [24]:
rows

[(1,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

In [25]:
rows[0][0]

1

In [26]:
v = np.frombuffer(rows[0][1], dtype=np.float64)
v

array([1.3, 3.5, 2.2, 0.9])

In [27]:
vectors = []
for row in rows:
  v = np.frombuffer(row[1], dtype=np.float64)
  vectors.append(v)

In [28]:
vectors

[array([1.3, 3.5, 2.2, 0.9]), array([2.8, 1.6, 3.8, 2.2])]

# Find nearest vector

In [29]:
q_vector = np.array([3.5, 2.2, 3.5, 5.5])

In [30]:
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) DESC
""", (sqlite3.Binary(q_vector.tobytes()),))

In [31]:
res = cursor.fetchone()

In [32]:
res

(b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?',)

In [33]:
np.frombuffer(res[0], dtype=np.float64)

array([1.3, 3.5, 2.2, 0.9])

## sqlite-vss

sqlite-vss is an SQLite extension designed for vector search, emphasizing local-first operations and easy integration into applications without external servers. Leveraging the Faiss library, it offers efficient similarity search and clustering capabilities.

In [1]:
!pip install sqlite-vss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00


In [5]:
import langchain_community.embeddings


In [7]:
pip install --upgrade langchain


In [8]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import SQLiteVSS
from langchain.document_loaders import TextLoader

In [28]:
loader = TextLoader(file_path='sample_data/state_union.txt')


In [29]:
try:
    loader = TextLoader(file_path='state_union.txt')
    # Your loading logic here
except FileNotFoundError as e:
    print(f"File not found: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


In [30]:
from langchain_community.document_loaders import TextLoader

# Adjust the path as needed
file_path = 'sample_data/state_union.txt'
loader = TextLoader(file_path=file_path)

try:
    documents = loader.lazy_load()
except FileNotFoundError as e:
    print(f"File not found: {e}")
except RuntimeError as e:
    print(f"Runtime error: {e}")


In [31]:
documents

<generator object TextLoader.lazy_load at 0x781d6fe1ad50>

In [32]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

In [33]:
texts

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.',
 'Groups of citizens blocking tanks with

In [34]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [35]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
# load it in sqlite-vss in a table named state_union.
# the db_file parameter is the name of the file you want
# as your sqlite database.

db = SQLiteVSS.from_texts(
    texts=texts,
    embedding=embedding_function,
    table="state_union",
    db_file="vss.db",
)

In [37]:
# query it
query = "What did the president say about Ketanji Brown Jackson"
data = db.similarity_search(query)

In [38]:
data

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Ame

In [39]:
# print results
data[0].page_content

'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'